In [ ]:
# Don't change the names
!mkdir msdir
!mkdir input
!mkdir output

In [ ]:
import os
import json
import stimela
import pkg_resources
from drasticali import drasticali
from IPython.display import Javascript
from ipywidgets import widgets, interact
from drasticali.drasticali import natural_keys

In [ ]:
def run_all(ev):
    # Runn all cells
    #display(Javascript('IPython.notebook.execute_cells_below()'))
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'
                       'IPython.notebook.get_selected_index()+4)'))

button = widgets.Button(description="Configure")
button.on_click(run_all)
display(button)

docker = widgets.Dropdown(options=['Docker', 'uDocker', 'Podman', 'Singularity'],
    value='uDocker',
    description='Container Tech'
)
display(docker)

from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit"
value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
DATA = dict()
MSDIR = 'msdir'
if docker.value in ['Docker']:
    recipe = stimela.Recipe('Make Pipeline', ms_dir=MSDIR, JOB_TYPE='docker')
elif docker.value in ['uDocker']:
    recipe = stimela.Recipe('Make Pipeline', ms_dir=MSDIR, JOB_TYPE='udocker')
elif docker.value in ['Podman']:
    recipe = stimela.Recipe('Make Pipeline', ms_dir=MSDIR, JOB_TYPE='podman')
elif docker.value in ['Singularity']:
    recipe = stimela.Recipe('Make Pipeline', ms_dir=MSDIR,
                            singularity_image_dir=os.environ["SINGULARITY_PULLFOLDER"])
cab_names = ['simms', 'simulator', 'calibrator', 'wsclean', 'ddfacet', 'cleanmask', 'casa_makemask',
             'casa_clean', 'lwimager', 'casa_tclean',
             'fitstool', 'pybdsm', 'aegean',]#stimela.CAB
def setup():
    for cab_name in cab_names:
        PARAM_FILE_PATH = pkg_resources.resource_filename('stimela', 'cargo/cab/{}'.format(cab_name))
        PARAM_FILE = '{}/parameters.json'.format(PARAM_FILE_PATH)

        with open(PARAM_FILE) as par:
            DATA[cab_name] = json.load(par)
setup()

In [ ]:
recipe_file = widgets.Text(
    value='',
    placeholder='recipe.json',
    description='Recipe File:',
    disabled=False
)
display(recipe_file)

In [ ]:
def run_all(ev):
    # Runn all cells
    #display(Javascript('IPython.notebook.execute_cells_below()'))
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'
                       'IPython.notebook.get_selected_index()+5)'))

button = widgets.Button(description="LOAD")
button.on_click(run_all)
display(button)

Select cab to use:

In [ ]:
data = drasticali.get_data(recipe_file.value)
recipe.jobs = []

In [ ]:
data_keys = data.keys()
data_keys.sort(key=natural_keys)
cab1 = widgets.Dropdown(
    options=data_keys,
    description='Task:',
    disabled=False)
display(cab1)

In [ ]:
def run_all(ev):
    # Runn all cells
    #display(Javascript('IPython.notebook.execute_cells_below()'))
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'
                       'IPython.notebook.get_selected_index()+3)'))

button = widgets.Button(description="SELECT")
button.on_click(run_all)
display(button)

In [ ]:
cab_value = cab1.value
saved_cab_params = data[cab_value]
num = cab_value.split('_')[0]
c = DATA[cab_value.split(num + '_')[-1]]
print(c['description'])
print('')
params = c['parameters']
PAR = dict()
for param in params:
    print("Name: " + str(param['name']))
    print("Info: " + str(param['info']))
    print("Type: " + str(param['dtype']))
    if 'default' in param:
        print "Default: " + str(param['default'])
    if 'required' in param:
        print "Required: " + str(param['required'])
    if param['dtype'] in ['str', 'file', 'list:str', 'list',#'list:int',
                          'list:float', 'list:list']:
        if param['name'] not in saved_cab_params:
            PAR[param['name']] = widgets.Text(value=None)
        else:
            PAR[param['name']] = widgets.Text(value=str(saved_cab_params[param['name']])
                                              if param['name'] in saved_cab_params else '')
    elif param['dtype'] in ['list:file']:# and param['default'] == None:
        if param['name'] in saved_cab_params:
            if type(saved_cab_params[param['name']]) is list:
                PAR[param['name']] = widgets.Textarea(value=str('\\n'.join(saved_cab_params[param['name']]))
                                                        if param['name'] in saved_cab_params else '')
            else:
                PAR[param['name']] = widgets.Text(value=saved_cab_params[param['name']])
        else:
            PAR[param['name']] = widgets.Text(value=None)
    elif type(param['dtype']) is list:
        for par in param['dtype']:
            if par in ['int']:
                try:
                    PAR[param['name']] = widgets.IntText(value=saved_cab_params[param['name']]
                                                         if param['name'] in saved_cab_params else None)
                except:
                    PAR[param['name']] = widgets.Text(value=saved_cab_params[param['name']]
                                                         if param['name'] in saved_cab_params else None)
            #elif par in ['float']:
            #    PAR[param['name']] = widgets.FloatText(value=saved_cab_params[param['name']]
            #                                   if param['name'] in saved_cab_params else None)
            else:
                PAR[param['name']] = widgets.Textarea(value=str(str(saved_cab_params[param['name']])
                                                      if param['name'] in saved_cab_params else ''))
    elif param['dtype'] in ['list:int']:
        if param['default'] == None:
            PAR[param['name']] = widgets.Text(value=None)
        else:
            PAR[param['name']] = widgets.IntText(value=saved_cab_params[param['name']]
                                             if param['name'] in saved_cab_params else None)
    elif param['dtype'] in ['int']:
        if param['name'] not in saved_cab_params:
        #if param['default'] == None:
            PAR[param['name']] = widgets.Text(value=None)
        else:
            PAR[param['name']] = widgets.IntText(value=int(saved_cab_params[param['name']])
                                                 if param['name'] in saved_cab_params else None)
    elif param['dtype'] in ['float']:
        if param['default'] == None:
            PAR[param['name']] = widgets.Text(value=None)
        else:
            PAR[param['name']] = widgets.FloatText(value=float(saved_cab_params[param['name']])
                                                   if param['name'] in saved_cab_params else None)
    elif param['dtype'] in ['bool']:
        if param['default'] == None:
            PAR[param['name']] = widgets.Text(value=None)
        else:
            PAR[param['name']] = widgets.Dropdown(options=[True, False],
                                                  value=bool(saved_cab_params[param['name']])
                                                  if param['name'] in saved_cab_params
                                                  else bool(param['default']))
    display(PAR[param['name']])
    print('')

In [ ]:
def run_all(ev):
    # Runn all cells
    #display(Javascript('IPython.notebook.execute_cells_below()'))
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'
                       'IPython.notebook.get_selected_index()+3)'))

button = widgets.Button(description="ADD")
button.on_click(run_all)
display(button)

cab_params = {}

In [ ]:
cab_value = cab1.value
saved_cab_params = data[cab_value]
cab_num = int(cab_value.split('_')[0])
cab_func_name = cab_value.split(num + '_')[-1]

c = DATA[cab_func_name]
params = c['parameters']
cab_func = getattr(drasticali, cab_func_name, None)
for key, val in PAR.items():
    key = str(key)
    if val.value in ['None', '', "''"]:
        continue
    if type(val.value) is str and '\\n' in val.value:
        cab_params[key] = val.value.split('\\n')
    ##temp fix since the type must be int
    elif key in ['sigma', 'Image-SidelobeSearchWindow', 'Deconv-NumRMSSamples', 'HMP-Taper',
                 'Noise-Sigma', 'threads', 'weighting-rank-filter-size', 'oversampling',
                 'start', 'width', 'nwlayers', 'nfacets']:
        cab_params[key] = int(val.value)
    elif key in ['stack']:
        if val.value in ['False', '', 'None']:
            cab_params[key] = False
        else:
            cab_params[key] = True
    elif type(val.value) is unicode:
        if key in ['sefd', 'padding']:
            cab_params[key] = float(val.value)
        elif '\\n' in str(val.value):
            cab_params[key] = val.value.split('\\n')
        else:
            cab_params[key] = str(val.value)
    else:
        cab_params[key] = val.value
cab_func(recipe, num=cab_num, parameters=cab_params, recipes_file=recipe_file.value)

In [ ]:
#%load_ext jupyternotify

def run_all(ev):
    # Runn all cells
    #display(Javascript('IPython.notebook.execute_cells_below()'))
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'
                       'IPython.notebook.get_selected_index()+2)'))

button = widgets.Button(description="RUN ALL")
button.on_click(run_all)
display(button)

In [ ]:
drasticali.run_all(recipe, recipe_file.value)